# Python Package Requirements

In [1]:
# Requirements:
## - Install the following python packages:
###  - `pip3 install cubes sqlalchemy flask`

# Create Schema and Load Data

In [2]:
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv
engine = create_engine("sqlite:///data.sqlite")
create_table_from_csv(engine,
                     "../data/Table_E.csv",
                     table_name="nyc_restaurants",
                     fields=[
                        ("name", "string"),
                        ("address", "string"),
                        ("zipcode", "string"),
                        ("cuisine", "string"),
                        ("price", "string"),
                        ("rating", "string"),
                        ("review_count", "integer"),
                        ("violation_code", "string"),
                        ("critical_flag", "string"),
                        ("grade", "string"),
                        ("median_household_income", "integer"),
                        ("median_real_estate_value", "integer"),
                        ("population_density", "integer"),
                        ("cost_of_living", "integer"),
                        ("population", "integer"),
                        ("neighborhood", "string"),
                        ("borough", "string")
                     ],
                     create_id=True)
print('Created schema and loaded data.')

Created schema and loaded data.


# Create OLAP Cube

In [3]:
from cubes import Workspace
workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")
workspace.import_model("olap_model.json")
browser = workspace.browser("nyc_restaurants")
cube = workspace.cube("nyc_restaurants")
print('Created OLAP cube.')

Created OLAP cube.


# Aggregation by borough

In [4]:
result = browser.aggregate(drilldown=["area:borough"])
print('Aggregation results:')
print("Borough, Number of Restaurants")
for record in result:
    print(record['area.borough'], ",",record['num_restaurants'])

Aggregation results:
Borough, Number of Restaurants
Bronx , 519
Brooklyn , 1846
Manhattan , 1821
Queens , 1078
Staten Island , 296


# Aggregation by neighborhood

In [5]:
result = browser.aggregate(drilldown=["area:neighborhood"])
print('Aggregation results:') 
print("Neighborhood, Number of Restaurants")
for record in result:
    print(record['area.neighborhood'], ",",record['num_restaurants'])

Aggregation results:
Neighborhood, Number of Restaurants
Bronx Park and Fordham , 103
Central Bronx , 23
High Bridge and Morrisania , 36
Hunts Point and Mott Haven , 38
Kingsbridge and Riverdale , 62
Northeast Bronx , 77
Southeast Bronx , 180
Borough Park , 136
Bushwick and Williamsburg , 172
Canarsie and Flatlands , 82
Central Brooklyn , 239
East New York and New Lots , 54
Flatbush , 105
Greenpoint , 207
Northwest Brooklyn , 442
Southern Brooklyn , 191
Southwest Brooklyn , 133
Sunset Park , 85
Battery Park , 3
Central Harlem , 88
Chelsea and Clinton , 260
East Harlem , 83
Gramercy Park and Murray Hill , 181
Greenwich Village and Soho , 273
Inwood and Washington Heights , 201
Lower East Side , 212
Lower Manhattan , 206
Upper East Side , 120
Upper West Side , 194
Central Queens , 31
Jamaica , 105
North Queens , 185
Northeast Queens , 67
Northwest Queens , 172
Southeast Queens , 71
Southwest Queens , 97
West Central Queens , 129
West Queens , 221
Mid-Island , 55
Port Richmond , 57
South 

# Aggregation by restaurant price type

In [6]:
result = browser.aggregate(drilldown=["price"])
print('Aggregation results:')
print("Price Type, Number of Restaurants")
for record in result:
    print(record['price'], ",", record['num_restaurants'])

Aggregation results:
Price Type, Number of Restaurants
$ , 2280
$$ , 2930
$$$ , 300
$$$$ , 50


# Aggregation by restaurant rating 

In [7]:
result = browser.aggregate(drilldown=["rating"])
print('Aggregation results:')
print("Rating, Number of Restaurants")
for record in result:
    print(record['rating'], ",", record['num_restaurants'])

Aggregation results:
Rating, Number of Restaurants
1.0 , 43
1.5 , 85
2.0 , 168
2.5 , 379
3.0 , 780
3.5 , 1424
4.0 , 1825
4.5 , 726
5.0 , 130


# Aggregation by critical health violation

In [8]:
result = browser.aggregate(drilldown=["critical_flag"])
print('Aggregation results:')
print("Health Violation, Number of Restaurants")
for record in result:
    print(record['critical_flag'], ",", record['num_restaurants'])

Aggregation results:
Health Violation, Number of Restaurants
Critical , 2991
Not Critical , 2569


# Aggregation by health violation grade

In [9]:
result = browser.aggregate(drilldown=["grade"])
print('Aggregation results:')
print("Health Violation Grade, Number of Restaurants")
for record in result:
    print(record['grade'], ",", record['num_restaurants'])

Aggregation results:
Health Violation Grade, Number of Restaurants
A , 4881
B , 422
C , 46
P , 10
Z , 201


# Slice on 'Critical' health violation by Borough

In [10]:
from cubes import PointCut
from cubes import Cell
cut = PointCut("critical_flag", ["Critical"])
cell = Cell(cube, [cut])
result = browser.aggregate(cell, drilldown=["area:borough"])
print('Aggregation results for grade "A" violation:')
print("Borough, Number of Restaurants")
for record in result:
    print(record['area.borough'], ",", record['num_restaurants'])

Aggregation results for grade "A" violation:
Borough, Number of Restaurants
Bronx , 281
Brooklyn , 977
Manhattan , 1007
Queens , 561
Staten Island , 165


# Dice on 'Critical' and Grade 'A' Health Violation by Borough

In [14]:
from cubes import PointCut
from cubes import Cell
cuts = [
    PointCut("grade", ["A"]),
    PointCut("critical_flag", ["Critical"])
]
cell = Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["area:borough"])
print('Aggregation results for "critical" and grade "A" violation:')
print("Borough, Number of Restaurants")
for record in result:
    print(record['area.borough'], ",", record['num_restaurants'])

Aggregation results for "critical" and grade "A" violation:
Borough, Number of Restaurants
Bronx , 230
Brooklyn , 821
Manhattan , 839
Queens , 500
Staten Island , 140


# Slice on 'Critical' Health Violation by Neighborhood

In [11]:
from cubes import PointCut
from cubes import Cell
cut = PointCut("critical_flag", ["Critical"])
cell = Cell(cube, [cut])
result = browser.aggregate(cell, drilldown=["area:neighborhood"])
print('Aggregation results for "critical" violation:')
print("Neighborhood, Number of Restaurants")
for record in result:
    print(record['area.neighborhood'], ",", record['num_restaurants'])

Aggregation results for "critical" violation:
Neighborhood, Number of Restaurants
Bronx Park and Fordham , 54
Central Bronx , 14
High Bridge and Morrisania , 16
Hunts Point and Mott Haven , 20
Kingsbridge and Riverdale , 39
Northeast Bronx , 43
Southeast Bronx , 95
Borough Park , 63
Bushwick and Williamsburg , 97
Canarsie and Flatlands , 41
Central Brooklyn , 124
East New York and New Lots , 28
Flatbush , 56
Greenpoint , 112
Northwest Brooklyn , 243
Southern Brooklyn , 95
Southwest Brooklyn , 77
Sunset Park , 41
Battery Park , 1
Central Harlem , 46
Chelsea and Clinton , 155
East Harlem , 52
Gramercy Park and Murray Hill , 94
Greenwich Village and Soho , 151
Inwood and Washington Heights , 99
Lower East Side , 115
Lower Manhattan , 107
Upper East Side , 76
Upper West Side , 111
Central Queens , 13
Jamaica , 54
North Queens , 95
Northeast Queens , 38
Northwest Queens , 91
Southeast Queens , 33
Southwest Queens , 50
West Central Queens , 58
West Queens , 129
Mid-Island , 34
Port Richmond 

# Dice on 'Critical' and Grade 'A' Health Violation by Neighborhood

In [15]:
from cubes import PointCut
from cubes import Cell
cuts = [
    PointCut("grade", ["A"]),
    PointCut("critical_flag", ["Critical"])
]
cell = Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["area:neighborhood"])
print('Aggregation results for "critical" and grade "A" violation:')
print("Neighborhood, Number of Restaurants")
for record in result:
    print(record['area.neighborhood'], ",", record['num_restaurants'])

Aggregation results for "critical" and grade "A" violation:
Neighborhood, Number of Restaurants
Bronx Park and Fordham , 46
Central Bronx , 10
High Bridge and Morrisania , 12
Hunts Point and Mott Haven , 17
Kingsbridge and Riverdale , 27
Northeast Bronx , 40
Southeast Bronx , 78
Borough Park , 48
Bushwick and Williamsburg , 76
Canarsie and Flatlands , 38
Central Brooklyn , 101
East New York and New Lots , 23
Flatbush , 49
Greenpoint , 100
Northwest Brooklyn , 207
Southern Brooklyn , 81
Southwest Brooklyn , 65
Sunset Park , 33
Central Harlem , 35
Chelsea and Clinton , 129
East Harlem , 43
Gramercy Park and Murray Hill , 79
Greenwich Village and Soho , 131
Inwood and Washington Heights , 71
Lower East Side , 93
Lower Manhattan , 89
Upper East Side , 69
Upper West Side , 100
Central Queens , 12
Jamaica , 48
North Queens , 85
Northeast Queens , 35
Northwest Queens , 82
Southeast Queens , 29
Southwest Queens , 42
West Central Queens , 52
West Queens , 115
Mid-Island , 31
Port Richmond , 26


# Slice on 'Critical' Health Violation by Rating

In [12]:
from cubes import PointCut
from cubes import Cell
cut = PointCut("critical_flag", ["Critical"])
cell = Cell(cube, [cut])
result = browser.aggregate(cell, drilldown=["rating"])
print('Aggregation results for "critical" violation:')
print("Rating, Number of Restaurants")
for record in result:
    print(record['rating'], ",", record['num_restaurants'])

Aggregation results for "critical" violation:
Rating, Number of Restaurants
1.0 , 21
1.5 , 46
2.0 , 82
2.5 , 211
3.0 , 414
3.5 , 793
4.0 , 979
4.5 , 378
5.0 , 67


# Dice on 'Critical' and Grade 'A' Health Violation by Rating

In [16]:
from cubes import PointCut
from cubes import Cell
cuts = [
    PointCut("grade", ["A"]),
    PointCut("critical_flag", ["Critical"])
]
cell = Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["rating"])
print('Aggregation results for "critical" and grade "A" violation:')
print("Rating, Number of Restaurants")
for record in result:
    print(record['rating'], ",", record['num_restaurants'])

Aggregation results for "critical" and grade "A" violation:
Rating, Number of Restaurants
1.0 , 19
1.5 , 40
2.0 , 73
2.5 , 175
3.0 , 333
3.5 , 666
4.0 , 836
4.5 , 329
5.0 , 59


# Slice on 'Critical' Health Violation by Price

In [13]:
from cubes import PointCut
from cubes import Cell
cut = PointCut("critical_flag", ["Critical"])
cell = Cell(cube, [cut])
result = browser.aggregate(cell, drilldown=["price"])
print('Aggregation results for "critical" violation:')
print("Rating, Number of Restaurants")
for record in result:
    print(record['price'], ",", record['num_restaurants'])

Aggregation results for "critical" violation:
Rating, Number of Restaurants
$ , 1215
$$ , 1597
$$$ , 155
$$$$ , 24


# Dice on 'Critical' and Grade 'A' Health Violation by Price

In [18]:
from cubes import PointCut
from cubes import Cell
cuts = [
    PointCut("grade", ["A"]),
    PointCut("critical_flag", ["Critical"])
]
cell = Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["price"])
print('Aggregation results for "critical" and grade "A" violation:')
print("Price, Number of Restaurants")
for record in result:
    print(record['price'], ",", record['num_restaurants'])

Aggregation results for "critical" and grade "A" violation:
Price, Number of Restaurants
$ , 1047
$$ , 1331
$$$ , 132
$$$$ , 20
